<a href="https://colab.research.google.com/github/tarasryb/algohouse_sdk/blob/master/algohouse_sdk_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tarasryb/algohouse_sdk.git

Cloning into 'algohouse_sdk'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 39 (delta 13), reused 32 (delta 9), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [3]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 271 kB 11.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for ta-lib: filename=TA_Lib-0.4.25-cp38-cp38-linux_x86_64.whl size=1820893 sha256=aad1ac4f52f1b46d0835be4b15c4f121898d1085ad73aae33ac3

In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import talib as ta
import pandas as pd

import algohouse_sdk.algohouse as ah

# Reference Data

In [5]:
ref_data = ah.get_reference_data()
ref_data.head(100)

,exchange,instruments
0,coinbase,"[00-USD, 1INCH-BTC, 1INCH-EUR, 1INCH-GBP, 1INC..."
1,kraken,"[1INCH/EUR, 1INCH/USD, AAVE/AUD, AAVE/ETH, AAV..."
2,cryptocom,"[1INCH_USD, 1INCH_USDT, AAVE_BTC, AAVE_USD, AA..."
3,gateio,"[10SET_USDT, 1ART_USDT, 1EARTH_USDT, 1INCH3L_U..."
4,okx,"[AAVE-ETH, AAVE-USDC, AAVE-USDT, ABT-USDT, ADA..."
5,bithumb,"[2KEY-USDT, 7UP-USDT, 8PAY-USDT, AAA-USDT, ABB..."
6,binanceus,"[1INCHUSD, 1INCHUSDT, AAVEUSD, AAVEUSDT, ACHUS..."
7,bybit,"[AAVEUSDT, ADAUSDT, ALGOUSDT, APEUSDT, APEXUSD..."
8,lbank,"[1INCH3L_USDT, 1INCH3S_USDT, 1INCH5L_USDT, 1IN..."
9,bitfinex,"[1INCH:USD, 1INCH:UST, AAABBB, AAVE:USD, AAVE:..."


In [6]:
bnc_ref = ref_data.loc[ref_data['exchange'] == 'binance/f']
print(list(bnc_ref['instruments']))

[['1000LUNCBUSD', '1000LUNCUSDT', '1000SHIBBUSD', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'AMBBUSD', 'ANCBUSD', 'ANKRUSDT', 'ANTUSDT', 'APEBUSD', 'APEUSDT', 'API3USDT', 'APTBUSD', 'APTUSDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUCTIONBUSD', 'AUDIOUSDT', 'AVAXBUSD', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLUEBIRDUSDT', 'BLZUSDT', 'BNBBUSD', 'BNBUSDT', 'BNXUSDT', 'BTCBUSD', 'BTCDOMUSDT', 'BTCUSDT', 'BTCUSDT_221230', 'C98USDT', 'CELOUSDT', 'CELRUSDT', 'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVXBUSD', 'CVXUSDT', 'DARUSDT', 'DASHUSDT', 'DEFIUSDT', 'DENTUSDT', 'DGBUSDT', 'DODOBUSD', 'DOGEBUSD', 'DOGEUSDT', 'DOTBUSD', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'EGLDUSDT', 'ENJUSDT', 'ENSUSDT', 'EOSUSDT', 'ETCBUSD', 'ETCUSDT', 'ETHBUSD', 'ETHUSDT', 'ETHUSDT_221230', 'FILBUSD', 'FILUSDT', 'FLMUSDT', 'FLOWUSDT', 'FOOTBALLUSDT'

# Settings

In [7]:
USER_EMAIL = 'tarasryb@gmail.com'
SIGNKEY = '9566c74d10037c4d7bbb0407d1e2c649'

exchange = 'binance/f'
instrument = 'APEBUSD'  #'1000LUNCBUSD'
start_time = '2022-11-01T00:00:00'
end_time = '2022-12-30T23:00:00'


# Trades Without Aggregation

In [ ]:
trades = ah.get_trades(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        to_time=end_time)
trades = trades.loc[trades['bs'] == 'S']
trades.tail(100)

In [26]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=trades['ts'], y=trades['price'], name="price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=trades['ts'], y=trades['volume'], name="volume"),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Price & Volume",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=True)

fig.show()

# Aggregated Trades

In [ ]:
trades_a = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        to_time=end_time,
                                        aggregation='1d')
trades_a.head(1000)

In [144]:
fig = go.Figure(data=[
    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price")
    ])
fig.update_layout(
    title = f"{instrument} Price",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False
)

fig.show()


# Technical Analysis

## Volume Indicators

In [73]:
window_start = '2022-11-20T00:00:00'
window_end = '2022-12-01T00:00:00'
df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=window_end,
                                        aggregation='1d')


In [74]:
ver_window_start = '2022-12-01T00:00:00'
ver_window_end = '2022-12-12T00:00:00'
ver_df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=ver_window_start,
                                        to_time=ver_window_end,
                                        aggregation='1d')


In [75]:
open = df['open']
high = df['high']
low = df['low']
close = df['close']
volume = df['volume']


### [AD - Chaikin A/D Line](https://www.investopedia.com/articles/trading/08/accumulation-distribution-line.asp)


In [85]:
ad = ta.AD(high, low, close, volume)
print(ad)

0    -3.332824e+06
1     1.089594e+06
2     1.022658e+07
3     9.382426e+06
4     8.552210e+06
5     7.298971e+06
6     1.470044e+07
7     2.000587e+07
8     8.221248e+06
9     4.542684e+06
10    7.400684e+06
dtype: float64


In [89]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ad, name="AD volume"),
    secondary_y=False,
)

fig.update_layout(
    title = f"{instrument} Chaikin A/D Line",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=False)
# fig.update_yaxes(title_text="<b>volume</b>", secondary_y=True)

fig.show()

### [ADOSC - Chaikin A/D Oscillator](https://www.investopedia.com/articles/active-trading/031914/understanding-chaikin-oscillator.asp)

In [82]:
adosc = ta.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
print(adosc)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
8               NaN
9     643804.201328
10    267090.533090
dtype: float64


In [90]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=adosc, name="Chaikin A/D Oscillator"),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Chaikin A/D Oscillator",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>Volume</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Chaikin A/D Oscillator</b>", secondary_y=True)

fig.show()

###  [OBV - On Balance Volume](https://www.investopedia.com/terms/o/onbalancevolume.asp)

In [91]:
ovb = ta.OBV(close, volume)
print(ovb)

0      6134754.9
1     15443480.6
2     25066005.8
3     33226154.2
4     39839256.6
5     36079540.5
6     45589306.1
7     63627772.7
8     38559904.8
9     52003384.9
10    68238002.6
dtype: float64


In [98]:
ver_price_df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=ver_window_end,
                                        aggregation='1d')


In [99]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

# fig.add_trace(
#     go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
#     secondary_y=False,
# )

fig.add_trace(
    go.Scatter(x=ver_price_df['ts'], y=ver_price_df['close'], name="verification price"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=ovb, name="OVB volume"),
    secondary_y=False,
)

fig.update_layout(
    title = f"{instrument} On Balance Volume",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price</b>", secondary_y=True)

fig.show()

### Basic Indicators
* [ADV - Average Daily Value Traded*](https://www.investopedia.com/terms/a/averagedailytradingvolume.asp)
* MA - Moving Average

In [164]:
idc = pd.DataFrame()
idc['ts'] = df['ts'] 
idc['volume'] = df['volume'] 
idc['ADV5d'] = df['volume'].rolling(window=5).mean().shift(1)
idc['ADV21d'] = df['volume'].rolling(window=21).mean().shift(1)
idc['MA1w'] = ta.MA(close, timeperiod=7, matype=0)
idc['MA3w'] = ta.MA(close, timeperiod=7*3, matype=0)
idc['VWAP'] = pd.Series(ta.WMA(close, timeperiod=24)) * df['volume']
idc.head()

,ts,volume,ADV5d,ADV21d,MA1w,MA3w,VWAP
0,2022-11-12,5057405.3,NaN,NaN,NaN,NaN,NaN
1,2022-11-13,5575186.0,NaN,NaN,NaN,NaN,NaN
2,2022-11-14,6385193.8,NaN,NaN,NaN,NaN,NaN
3,2022-11-15,4064182.4,NaN,NaN,NaN,NaN,NaN
4,2022-11-16,3957640.3,NaN,NaN,NaN,NaN,NaN


In [171]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    go.Candlestick(x=df['ts'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['volume'], name="volume", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['ADV5d'], name="ADV 5d", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['ADV21d'], name="ADV 21d", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['MA1w'], name="MA 1w", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['MA3w'], name="MA 3w", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['VWAP'], name="VWAP", mode = 'lines+markers'),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Basic Indicators",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>ATR</b>", secondary_y=True)

fig.show()

## Volatility Indicators

In [121]:
window_start = '2022-11-01T00:00:00'
window_end = '2022-12-12T00:00:00'
df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=window_end,
                                        aggregation='1d')
open = df['open']
high = df['high']
low = df['low']
close = df['close']
volume = df['volume']


### [ATR - Average True Range](https://www.investopedia.com/terms/a/atr.asp)

In [122]:
atr = ta.ATR(high, low, close, timeperiod=14)
# print(atr)

In [143]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=atr, name="ATR", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} Average True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>ATR</b>", secondary_y=True)

fig.show()

### [NATR - Normalized Average True Range](https://support.atas.net/en/knowledge-bases/2/articles/43436-atr-normalized)

In [127]:
natr = ta.NATR(high, low, close, timeperiod=14)
# print(natr)

In [142]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=natr, name="NATR", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} Normalized Average True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>NATR</b>", secondary_y=True)

fig.show()

### [TRANGE - True Range](https://support.atas.net/en/knowledge-bases/2/articles/45183-true-range)

In [132]:
trange = ta.TRANGE(high, low, close)
# print(trange)

In [141]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=trange, name="TRANGE", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>TRANGE</b>", secondary_y=True)

fig.show()

## Price Transform Functions
* AVGPRICE - Average Price
* MEDPRICE - Median Price
* TYPPRICE - Typical Price
* WCLPRICE - Weighted Close Price


In [135]:
avgprice = ta.AVGPRICE(open, high, low, close)
medprice = ta.MEDPRICE(high, low)
typprice = ta.TYPPRICE(high, low, close)
wclprice = ta.WCLPRICE(high, low, close)

In [140]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=avgprice, name="AVGPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=medprice, name="MEDPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=typprice, name="TYPPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=wclprice, name="WCLPRICE", mode = 'lines+markers'),
    secondary_y=False,
)


fig.update_layout(
    title = f"{instrument} Price Transform Functions",
    autosize=False,
    width=1200,
    height=600,
    template='plotly_white',
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
# fig.update_yaxes(title_text="<b>TRANGE</b>", secondary_y=True)

fig.show()